In [1]:
import pandas as pd
import pathlib
import duckdb
from IPython.display import display, HTML
import unidecode
import textwrap

# --- Parâmetros de Entrada ---
PASTA_PROCESSED = pathlib.Path("dados_catalogo") / "processed"
ARQUIVO_DE_ENTRADA = PASTA_PROCESSED / 'dados_consolidados_filtrados.parquet'
#ARQUIVO_DE_SAIDA = PASTA_PROCESSED / 'tabela_dimensao_splink.parquet'
ARQUIVO_DE_SAIDA_LIMPO = PASTA_PROCESSED / 'tabela_dimensao_limpa_para_splink.parquet'

# --- Regras de Negócio Já Definidas ---
# Semanas a serem EXCLUÍDAS da análise de metadados, com base na nossa investigação.
SEMANAS_A_EXCLUIR = ['2024-10-25', '2024-11-21']

print("--- Etapa 3: Análise Exploratória e Preparação da Tabela Dimensão ---")
print("Bloco 1: Configurações carregadas.")

--- Etapa 3: Análise Exploratória e Preparação da Tabela Dimensão ---
Bloco 1: Configurações carregadas.


In [2]:
print("\n" + "="*80)
print(" ANÁLISE EXPLORATÓRIA 1: COMPLETUDE DAS COLUNAS")
print("="*80)
print("Analisando o percentual de preenchimento (não-nulos) de cada coluna...")

try:
    con = duckdb.connect(database=':memory:')
    df_schema = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}');").fetch_df()
    colunas = df_schema['column_name'].tolist()
    
    metricas_completude = []
    for col in colunas:
        query = f"SELECT 100.0 * COUNT(\"{col}\") / COUNT(*) FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}');"
        pct_preenchimento = con.execute(query).fetchone()[0]
        metricas_completude.append({"Coluna": col, "Preenchimento (%)": pct_preenchimento})
        
    df_completude = pd.DataFrame(metricas_completude).sort_values(by="Preenchimento (%)", ascending=False)

    display(HTML("<h4>Relatório de Completude de Colunas</h4>"))
    display(df_completude.style.format({'Preenchimento (%)': '{:.2f}%'})
            .bar(subset=['Preenchimento (%)'], color='#5DADE2', vmin=0, vmax=100))
    
    con.close()
except Exception as e:
    print(f"ERRO durante a análise de completude: {e}")
    if 'con' in locals(): con.close()


 ANÁLISE EXPLORATÓRIA 1: COMPLETUDE DAS COLUNAS
Analisando o percentual de preenchimento (não-nulos) de cada coluna...


,Coluna,Preenchimento (%)
0,BB_UID,100.00%
1,Platform_Content_ID,100.00%
2,BB_Hash_Unique,100.00%
3,Platform_Name,100.00%
4,Platform_Country,100.00%
5,Package,100.00%
7,Type,100.00%
8,Deeplink,100.00%
10,Is_Original,100.00%
11,Is_Exclusive,100.00%


In [3]:
print("\n" + "="*80)
print(" ANÁLISE EXPLORATÓRIA 2: ESTABILIDADE DAS COLUNAS")
print("="*80)
print("Analisando a variabilidade dos metadados por BB_UID...")

try:
    con = duckdb.connect(database=':memory:')
    
    # Pega todas as colunas exceto as de controle
    colunas_para_analise = [col for col in colunas if col not in ['BB_UID', 'data_ref', 'arquivo_origem']]
    
    # Monta a query para contar valores distintos por UID para cada coluna
    agregacoes_distinct = [f'COUNT(DISTINCT \"{col}\") AS {col}' for col in colunas_para_analise]
    query_estabilidade = f"""
    SELECT
        {', '.join(agregacoes_distinct)}
    FROM
        read_parquet('{str(ARQUIVO_DE_ENTRADA)}')
    GROUP BY
        "BB_UID"
    """
    
    df_contagens = con.execute(query_estabilidade).fetch_df()
    
    # Calcula o percentual de UIDs que possuem mais de 1 valor para cada coluna
    metricas_estabilidade = []
    for col in df_contagens.columns:
        uids_inconsistentes = (df_contagens[col] > 1).sum()
        pct_inconsistente = 100.0 * uids_inconsistentes / len(df_contagens)
        metricas_estabilidade.append({"Coluna": col, "% de UIDs com >1 Valor": pct_inconsistente})
        
    df_estabilidade = pd.DataFrame(metricas_estabilidade).sort_values(by="% de UIDs com >1 Valor", ascending=True)

    display(HTML("<h4>Relatório de Estabilidade de Colunas</h4><p>Quanto menor o percentual, mais estável (confiável) é a coluna.</p>"))
    display(df_estabilidade.style.format({'% de UIDs com >1 Valor': '{:.2f}%'})
            .background_gradient(cmap='Greens_r', subset=['% de UIDs com >1 Valor']))

    con.close()
except Exception as e:
    print(f"ERRO durante a análise de estabilidade: {e}")
    if 'con' in locals(): con.close()


 ANÁLISE EXPLORATÓRIA 2: ESTABILIDADE DAS COLUNAS
Analisando a variabilidade dos metadados por BB_UID...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Coluna,% de UIDs com >1 Valor
40,BB_Launch_Date,0.00%
38,IMDb_ID,0.00%
36,TMDB_ID,0.01%
37,TVDB_ID,0.19%
21,BB_Year,0.73%
9,Is_Original,1.37%
26,BB_Primary_Country,2.61%
39,Season_Numbers,3.37%
8,Seasons,4.55%
27,BB_Countries,5.91%


In [4]:
# ==============================================================================
# BLOCO DE DECISÃO: Sua seleção de colunas foi aplicada.
# ==============================================================================
# A lista abaixo foi criada a partir da sua análise e da imagem fornecida.
# Estas são as colunas que irão compor a nossa Tabela Dimensão.

COLUNAS_PARA_SPLINK = [
    # --- Identificação Essencial ---
    "BB_UID",
    "Type",
    
    # --- Metadados Principais da Obra (FICA) ---
    "BB_Title",
    "BB_Year",
    "BB_Duration",
    "BB_Genres",
    "BB_Directors",
    "BB_Original_Title",
    "IMDb_ID",
    "BB_Countries",
    "BB_Primary_Country",
    "BB_Production_Companies",
    "BB_Primary_Company",
    "TMDB_ID",
    "BB_Cast"
]

print("As seguintes colunas foram selecionadas para a criação da Tabela Dimensão:")
print(textwrap.fill(", ".join(COLUNAS_PARA_SPLINK), 80))

As seguintes colunas foram selecionadas para a criação da Tabela Dimensão:
BB_UID, Type, BB_Title, BB_Year, BB_Duration, BB_Genres, BB_Directors,
BB_Original_Title, IMDb_ID, BB_Countries, BB_Primary_Country,
BB_Production_Companies, BB_Primary_Company, TMDB_ID, BB_Cast


In [ ]:

# --- Funções Auxiliares de Limpeza (pré-Splink) ---
def normalizar_string(texto):
    """Aplica limpeza segura em um campo de texto: minúsculas, remove acentos e espaços extras."""
    if texto is None or pd.isna(texto):
        return None
    texto = str(texto).lower()
    texto = unidecode.unidecode(texto)
    return texto.strip()

def normalizar_lista_em_string(texto, separador=','):
    """Normaliza uma string que representa uma lista (ex: elenco, diretores)."""
    if texto is None or pd.isna(texto):
        return None
    nomes = str(texto).split(separador)
    nomes_limpos = sorted([normalizar_string(nome) for nome in nomes if str(nome).strip()])
    return separador.join(nomes_limpos) if nomes_limpos else None

In [5]:
# ==============================================================================
# BLOCO 5: EXECUÇÃO FINAL - Construção da Tabela Dimensão
# ==============================================================================
print("\n" + "="*80)
print(" EXECUÇÃO FINAL: CONSTRUÇÃO DA TABELA DIMENSÃO")
print("="*80)

try:
    con = duckdb.connect(database=':memory:')
    
    # A query agora usa a lista COLUNAS_PARA_SPLINK que você definiu no bloco anterior
    # e a lista de semanas que já havíamos definido
    colunas_selecionadas_sql = ", ".join([f'"{c}"' for c in COLUNAS_PARA_SPLINK + ['data_ref']])
    
    query_crunch_temporal = f"""
    WITH dados_filtrados AS (
        SELECT {colunas_selecionadas_sql}
        FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}')
        WHERE data_ref NOT IN {tuple(SEMANAS_A_EXCLUIR) if SEMANAS_A_EXCLUIR else '()'}
    ),
    ranking_por_data AS (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY "BB_UID" ORDER BY data_ref DESC) as rn
        FROM dados_filtrados
    )
    SELECT {', '.join([f'"{c}"' for c in COLUNAS_PARA_SPLINK])}
    FROM ranking_por_data
    WHERE rn = 1
    """

    print("\nExecutando o 'crunch temporal' para criar os perfis únicos...")
    con.execute(f"COPY ({query_crunch_temporal}) TO '{str(ARQUIVO_DE_SAIDA)}' (FORMAT PARQUET, COMPRESSION 'snappy');")
    print(f"\n✔ Tabela dimensão criada com sucesso em: '{ARQUIVO_DE_SAIDA}'")
    
    df_resultado = con.execute(f"SELECT COUNT(*) as total_perfis_unicos FROM '{ARQUIVO_DE_SAIDA}'").fetch_df()
    total_perfis = df_resultado['total_perfis_unicos'].iloc[0]
    
    print(f"\n--- Resumo da Operação ---")
    print(f"  - Perfis únicos na Tabela Dimensão final: {total_perfis:,}")
    
    display(HTML("<h4>Amostra da Tabela Dimensão Gerada:</h4>"))
    display(con.execute(f"SELECT * FROM '{ARQUIVO_DE_SAIDA}' LIMIT 10").fetch_df())
    
    con.close()
    print("\nProcesso concluído.")

except Exception as e:
    print(f"\nOcorreu um erro durante o processo: {e}")
    if 'con' in locals():
        con.close()


 EXECUÇÃO FINAL: CONSTRUÇÃO DA TABELA DIMENSÃO

Executando o 'crunch temporal' para criar os perfis únicos...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✔ Tabela dimensão criada com sucesso em: 'dados_catalogo\processed\tabela_dimensao_splink.parquet'

--- Resumo da Operação ---
  - Perfis únicos na Tabela Dimensão final: 225,117


,BB_UID,Type,BB_Title,BB_Year,BB_Duration,BB_Genres,BB_Directors,BB_Original_Title,IMDb_ID,BB_Countries,BB_Primary_Country,BB_Production_Companies,BB_Primary_Company,TMDB_ID,BB_Cast
0,445d312e8a15c1e5b6d83a9bd66879a4,Series,I Am Not Okay with This,2020.0,23.0,"Sci-fi, Comedy, Fantasy, Drama",Jonathan Entwistle,I Am Not Okay with This,tt9446688,US,US,"21 Laps Entertainment, Ceremony Pictures, Rain...",21 Laps Entertainment,90260.0,"Sophia Lillis, Wyatt Oleff, Sofia Bryant, Kath..."
1,449425b8c5e58b376a6b4243a243cb3b,Series,Alan Gilbert and Wenzel Fuchs with Mozart’s Cl...,2018.0,NaN,None,Alan Gilbert,None,None,None,None,None,None,NaN,None
2,449e9b567bda9d6972bc3085288bb22f,Movie,Azadi,2025.0,129.0,"Drama, Thriller",Joe George,Azadi,tt27494358,IN,IN,Little Crew Productions,Little Crew Productions,1115022.0,"Sreenath Bhasi, Raveena Ravi, Lal, Salim Kumar..."
3,44bb2a4e73e311a1e88fc4227a1e3da0,Movie,Austrália v Brasil | Disputa do 3º lugar | FIF...,2001.0,NaN,None,None,None,None,None,None,None,None,NaN,None
4,44d59e5e804afee401af01ae22f0e835,Movie,On Our Own,2010.0,21.0,"Documentary, Short",Camiel Zwart,On Our Own,tt1764532,NL,NL,None,None,660350.0,None
5,4528ba9db05a36a47d9365bad4038fa5,Movie,Free Jazz in Kongressaal Munich: Cecil Taylor,1984.0,55.0,None,János Darvas,None,None,None,None,None,None,NaN,None
6,454134cb25befa1b10a8f9dcd656aa2a,Movie,Ding Dong Bell - Pop Rock Style,2016.0,1.0,None,"A,N",None,None,None,None,None,None,NaN,None
7,4575c4b5a775cc2614bb4c0c4f416b2c,Movie,LINA WERTMÜLLER,2017.0,24.0,None,Serena Ucelli Di Nemi,None,None,None,None,None,None,NaN,None
8,45a45e5aa30f69307dced5fc00da45b3,Series,Legend of the Dragon,2006.0,24.0,"Animation, Action, Adventure, Fantasy, Sci-fi",Tom Tataranowicz,Legend of the Dragon,tt0776469,"HK, US, GB, DE",GB,"Sony Pictures Entertainment, Bkn International",Sony Pictures Entertainment,16815.0,"Marc Silk, Alan Marriott, Gary Martin, Larissa..."
9,46368e888c1363d2dedd23bc87c89409,Movie,DET @ CHI,2014.0,130.0,None,None,None,None,None,None,None,None,NaN,None



Processo concluído.
